In [2]:
# Notebook d'Analyse:

In [3]:
# coding: utf-8
import pandas as pd
import numpy as np
import math

In [4]:
# read the csv file
df = pd.read_csv("movie_metadata.csv", sep=',',low_memory=False)

In [6]:
df.head(5).style

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723,178,0,855,Joel David Moore,1000,7.60506e+08,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_tt_tt_1,3054,English,USA,PG-13,2.37e+08,2009,936,7.9,1.78,33000
1,Color,Gore Verbinski,302,169,563,1000,Orlando Bloom,40000,3.09404e+08,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,471220,48350,Jack Davenport,0,goddess|marriage ceremony|marriage proposal|pirate|singapore,http://www.imdb.com/title/tt0449088/?ref_=fn_tt_tt_1,1238,English,USA,PG-13,3e+08,2007,5000,7.1,2.35,0
2,Color,Sam Mendes,602,148,0,161,Rory Kinnear,11000,2.00074e+08,Action|Adventure|Thriller,Christoph Waltz,Spectre,275868,11700,Stephanie Sigman,1,bomb|espionage|sequel|spy|terrorist,http://www.imdb.com/title/tt2379713/?ref_=fn_tt_tt_1,994,English,UK,PG-13,2.45e+08,2015,393,6.8,2.35,85000
3,Color,Christopher Nolan,813,164,22000,23000,Christian Bale,27000,4.48131e+08,Action|Thriller,Tom Hardy,The Dark Knight Rises,1144337,106759,Joseph Gordon-Levitt,0,deception|imprisonment|lawlessness|police officer|terrorist plot,http://www.imdb.com/title/tt1345836/?ref_=fn_tt_tt_1,2701,English,USA,PG-13,2.5e+08,2012,23000,8.5,2.35,164000
4,nan,Doug Walker,nan,nan,131,nan,Rob Walker,131,nan,Documentary,Doug Walker,Star Wars: Episode VII - The Force Awakens,8,143,nan,0,nan,http://www.imdb.com/title/tt5289954/?ref_=fn_tt_tt_1,nan,nan,nan,nan,nan,nan,12,7.1,nan,0


Features qui semblent les plus pertinentes: 
+ genres: genres auxquels appartient le film
+ title_year: année de sortie du film
+ imdb_score: note pondérée calculée selon la formule d'IMDB prenant en compte le nombre de votants. (https://help.imdb.com/article/imdb/track-movies-tv/weighted-average-ratings/GWT2DSBYVT2F25SK?ref_=cons_tt_rt_wtavg#)
+ director_name: nom du réalisateur

In [11]:
df.describe()

,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_1_facebook_likes,gross,num_voted_users,cast_total_facebook_likes,facenumber_in_poster,num_user_for_reviews,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
count,4993.000000,5028.000000,4939.000000,5020.000000,5036.000000,4.159000e+03,5.043000e+03,5043.000000,5030.000000,5022.000000,4.551000e+03,4935.000000,5030.000000,5043.000000,4714.000000,5043.000000
mean,140.194272,107.201074,686.509212,645.009761,6560.047061,4.846841e+07,8.366816e+04,9699.063851,1.371173,272.770808,3.975262e+07,2002.470517,1651.754473,6.442138,2.220403,7525.964505
std,121.601675,25.197441,2813.328607,1665.041728,15020.759120,6.845299e+07,1.384853e+05,18163.799124,2.013576,377.982886,2.061149e+08,12.474599,4042.438863,1.125116,1.385113,19320.445110
min,1.000000,7.000000,0.000000,0.000000,0.000000,1.620000e+02,5.000000e+00,0.000000,0.000000,1.000000,2.180000e+02,1916.000000,0.000000,1.600000,1.180000,0.000000
25%,50.000000,93.000000,7.000000,133.000000,614.000000,5.340988e+06,8.593500e+03,1411.000000,0.000000,65.000000,6.000000e+06,1999.000000,281.000000,5.800000,1.850000,0.000000
50%,110.000000,103.000000,49.000000,371.500000,988.000000,2.551750e+07,3.435900e+04,3090.000000,1.000000,156.000000,2.000000e+07,2005.000000,595.000000,6.600000,2.350000,166.000000
75%,195.000000,118.000000,194.500000,636.000000,11000.000000,6.230944e+07,9.630900e+04,13756.500000,2.000000,326.000000,4.500000e+07,2011.000000,918.000000,7.200000,2.350000,3000.000000
max,813.000000,511.000000,23000.000000,23000.000000,640000.000000,7.605058e+08,1.689764e+06,656730.000000,43.000000,5060.000000,1.221550e+10,2016.000000,137000.000000,9.500000,16.000000,349000.000000


In [12]:
df.columns

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes'],
      dtype='object')

Cette base de données est essentiellement centrée sur des films en anglais. Bien que la plupart de ces films existent dans d'autres langues. Nous allons donc supposer que les utilisateurs sont anglophones.

In [13]:
def get_percentage_missing(dframe):
    num = dframe.isnull().sum()
    den = len(dframe)
    return (round(num/den, 2)*100)

print(get_percentage_missing(df).sort_values())

color                         0.0
imdb_score                    0.0
actor_2_facebook_likes        0.0
country                       0.0
language                      0.0
num_user_for_reviews          0.0
movie_imdb_link               0.0
facenumber_in_poster          0.0
actor_3_name                  0.0
num_voted_users               0.0
movie_title                   0.0
cast_total_facebook_likes     0.0
genres                        0.0
actor_1_facebook_likes        0.0
actor_2_name                  0.0
actor_3_facebook_likes        0.0
duration                      0.0
actor_1_name                  0.0
movie_facebook_likes          0.0
num_critic_for_reviews        1.0
director_facebook_likes       2.0
title_year                    2.0
director_name                 2.0
plot_keywords                 3.0
content_rating                6.0
aspect_ratio                  7.0
budget                       10.0
gross                        18.0
dtype: float64


#### On va transformer les noms d'acteurs et de réalisateurs dans notre dataset pour simplifier leur traitement

In [14]:
def simplify_name(n):
    if isinstance(n, list):
        return [str.lower(i.replace(" ", "")) for i in n]
    else:
        #Check if director/actor exists. If not, return empty string
        if isinstance(n, str):
            return str.lower(n.replace(" ", ""))
        else:
            return ''

In [15]:
names=['director_name','actor_2_name','actor_3_name','actor_1_name']
for name in names:
    df[name]=df[name].apply(simplify_name)

#### On transforme les genres en listes

In [16]:
f=lambda x: x.lower().split("|")

df['genres']=df['genres'].apply(f)


In [21]:
g=lambda x: x.split('|') if isinstance(x, str) else x

df['plot_keywords']=df['plot_keywords'].apply(g)



In [22]:
df.head(5).style

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,jamescameron,723,178,0,855,joeldavidmoore,1000,7.60506e+08,"['action', 'adventure', 'fantasy', 'sci-fi']",cchpounder,Avatar,886204,4834,wesstudi,0,"['avatar', 'future', 'marine', 'native', 'paraplegic']",http://www.imdb.com/title/tt0499549/?ref_=fn_tt_tt_1,3054,English,USA,PG-13,2.37e+08,2009,936,7.9,1.78,33000
1,Color,goreverbinski,302,169,563,1000,orlandobloom,40000,3.09404e+08,"['action', 'adventure', 'fantasy']",johnnydepp,Pirates of the Caribbean: At World's End,471220,48350,jackdavenport,0,"['goddess', 'marriage ceremony', 'marriage proposal', 'pirate', 'singapore']",http://www.imdb.com/title/tt0449088/?ref_=fn_tt_tt_1,1238,English,USA,PG-13,3e+08,2007,5000,7.1,2.35,0
2,Color,sammendes,602,148,0,161,rorykinnear,11000,2.00074e+08,"['action', 'adventure', 'thriller']",christophwaltz,Spectre,275868,11700,stephaniesigman,1,"['bomb', 'espionage', 'sequel', 'spy', 'terrorist']",http://www.imdb.com/title/tt2379713/?ref_=fn_tt_tt_1,994,English,UK,PG-13,2.45e+08,2015,393,6.8,2.35,85000
3,Color,christophernolan,813,164,22000,23000,christianbale,27000,4.48131e+08,"['action', 'thriller']",tomhardy,The Dark Knight Rises,1144337,106759,josephgordon-levitt,0,"['deception', 'imprisonment', 'lawlessness', 'police officer', 'terrorist plot']",http://www.imdb.com/title/tt1345836/?ref_=fn_tt_tt_1,2701,English,USA,PG-13,2.5e+08,2012,23000,8.5,2.35,164000
4,nan,dougwalker,nan,nan,131,nan,robwalker,131,nan,['documentary'],dougwalker,Star Wars: Episode VII - The Force Awakens,8,143,,0,nan,http://www.imdb.com/title/tt5289954/?ref_=fn_tt_tt_1,nan,nan,nan,nan,nan,nan,12,7.1,nan,0


In [19]:
df.to_csv('movie_db.csv',sep='\t',index=False,encoding='utf-8')